### Deep Learning


In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

import mplcairo
import matplotlib as mpl
mpl.use("module://mplcairo.qt")
import matplotlib.pyplot as plt
from utils import *

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
gold_df = pd.read_csv("./data/data.tsv", sep='\t')
gold_df = gold_df[['tweet', 'sentiment']]
# drop records where sentiment is none or mixed
gold_df = gold_df.drop(gold_df[gold_df.sentiment == 'mixed'].index)
gold_df = gold_df.drop(gold_df[gold_df.sentiment == 'none'].index)
gold_df.head()

,tweet,sentiment
2,ناشرینترین کەس ئەو کەسەیە کە هەڵەکانت تۆمار دە...,negative
3,ئیمڕۆ هاوکارەکەم وتی تەک مناڵەکانا چووگین بۆ د...,negative
4,بە خۆر بڵێ کیس و شت نەکا وە با، مانگیش شەوانە ...,neutral
5,ئیشێ من ئەوەیە هەموو ڕۆژێ لە خەوێ‌ هەستم برێم:...,negative
6,ئەرێ ئێوەش ئاوان؟ لە ماڵەوە بێزارن و و کە دەچی...,negative


In [6]:
silver_df = pd.read_csv('./data/silver_data.csv')
neu_aug = silver_df[silver_df.sentiment == 'neutral'].sample(700 - 254)
pos_aug = silver_df[silver_df.sentiment == 'positive'].sample(700 - 292)
neg_aug = silver_df[silver_df.sentiment == 'negative'].sample(700 - 639)
df = pd.concat([neu_aug, pos_aug, neg_aug, gold_df], ignore_index=True)

In [7]:
# create a new df from gold_df index starts from 0
# df = gold_df.reset_index(drop=True)

In [8]:
df['cleaned_text'] = df['tweet'].apply(lambda x:clean_text(x, lemma=True, emoji=False))

In [9]:
df.head()

,tweet,sentiment,cleaned_text
0,حالیەن عیدەی من بەمیشکم دەری بەتاڵم😁,neutral,حالیەن عیدەی من بەمیشکم دەری بەتاڵم😁
1,ئەوە قسەی دڵی بەلووچ نیە بەڵام ، لەبەر ئەوە سا...,neutral,ئەوە قسە دڵ بەلووچ نیە بەڵا لەبەر ئەوە سانسوور...
2,دەڵێن مللەتی ئیران. کەنداو؛ کەنداوی فارسیە پشی...,neutral,دەڵ مللەت ئیران کەنداو؛ کەنداوی فارسیە پشیلە پ...
3,دەی کاکە تو ئەۆ خۆدایە ئەگەر ئاگاداری لە بنەما...,neutral,دەی کاکە تو ئەۆ خۆدایە ئەگەر ئاگاداری لە بنەما...
4,سیمینار لەسەر خەو و خواردن دەکا بەو سعات سێیەی🥲,neutral,سیمین لەسەر خەواندن و خواردن دەکا بەو سعات سێیەی🥲


In [10]:
df.sentiment.value_counts()

neutral     700
positive    700
negative    700
Name: sentiment, dtype: int64

In [11]:
RS = 42

In [12]:
# if you want to balance the classes change the following variable
num_of_categories = 1700

shuffled = df.reindex(np.random.permutation(df.index))
pos = shuffled[shuffled['sentiment'] == 'positive'][:num_of_categories]
neg = shuffled[shuffled['sentiment'] == 'negative'][:num_of_categories]
neu = shuffled[shuffled['sentiment'] == 'neutral'][:num_of_categories]
concated = pd.concat([pos, neg, neu], ignore_index=True)
concated['sentiment'] = concated['sentiment'].replace({'positive': 1, 'negative': -1, 'neutral': 0})
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))

In [13]:


print(concated['sentiment'][:10])
labels = to_categorical(concated['sentiment'], num_classes=3)
print(labels[:10])
if 'sentiment' in concated.keys():
    concated.drop(['sentiment'], axis=1)

52      1
1090   -1
546     1
1996    0
1740    0
1989    0
106     1
1432    0
1796    0
131     1
Name: sentiment, dtype: int64
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [14]:
n_most_common_words = 10000
max_len = 300
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['cleaned_text'].values)
sequences = tokenizer.texts_to_sequences(concated['cleaned_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
vocab_size = len(word_index)

X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

Found 8907 unique tokens.


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.2, random_state=42)
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

((1680, 300), (1680, 3), (420, 300), (420, 3))


In [16]:
epochs = 10
emb_dim = 100
batch_size = 256

In [17]:
import keras.backend as K


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [18]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size+1, output_dim=emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.1)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m, precision_m, recall_m])
print(model.summary())

2023-03-09 19:06:46.835582: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.017744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.018385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.020266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

2023-03-09 19:06:47.022236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.022708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.022987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.807152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 19:06:47.807480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          890800    
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 100)         0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 300, 128)         84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                        

In [95]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=8, min_delta=0.0001)])

Epoch 1/10
3/3 [==============================] - 15s 2s/step - loss: 1.0886 - accuracy: 0.4459 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.0487 - val_accuracy: 0.5842 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 5s 2s/step - loss: 1.0423 - accuracy: 0.5356 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.9888 - val_accuracy: 0.5842 - val_f1_m: 0.0503 - val_precision_m: 0.5556 - val_recall_m: 0.0263
Epoch 3/10
3/3 [==============================] - 5s 2s/step - loss: 1.0090 - accuracy: 0.5356 - f1_m: 0.4083 - precision_m: 0.5585 - recall_m: 0.3811 - val_loss: 0.9731 - val_accuracy: 0.5842 - val_f1_m: 0.5842 - val_precision_m: 0.5842 - val_recall_m: 0.5842
Epoch 4/10
3/3 [==============================] - 5s 2s/step - loss: 1.0146 - accuracy: 0.5356 - f1_m: 0.5357 - precision_m: 0.5357 - recall_m: 0.5357 - val_loss: 0.9700 - val_accu

In [96]:
accr = model.evaluate(X_test, y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  F1:{:0.3f}\n  Precision: {:0.3f}\n  Recall: {:0.3f} '
      .format(accr[0], accr[1], accr[2], accr[3], accr[4]))
# note different emb dimension only increased training acc

8/8 [==============================] - 1s 109ms/step - loss: 1.0217 - accuracy: 0.5148 - f1_m: 0.5095 - precision_m: 0.5141 - recall_m: 0.5051
Test set
  Loss: 1.022
  Accuracy: 0.515
  F1:0.509
  Precision: 0.514
  Recall: 0.505 


### Inference

In [62]:
samples = ["جوان 🥀🌟 ئەوەیە بە ناو خەڵکدا بڕۆی بۆنی ئەخلاقت لێ بێت.🙂", "بە ڕاستی ژیان ناخۆشە لێرە🖤", "زەوی تەختە نەک خڕ گۆشە"]

for sample in samples:
    seq = tokenizer.texts_to_sequences([sample])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded, verbose=0)
    labels = ['neutral', 'positive', 'negative']
    print(sample, labels[np.argmax(pred)])

جوان 🥀🌟 ئەوەیە بە ناو خەڵکدا بڕۆی بۆنی ئەخلاقت لێ بێت.🙂 positive
بە ڕاستی ژیان ناخۆشە لێرە🖤 positive
زەوی تەختە نەک خڕ گۆشە neutral



# Reference

   - https://www.kaggle.com/code/tanulsingh077/twitter-sentiment-extaction-analysis-eda-and-model
   - https://www.kaggle.com/code/ngyptr/multi-class-classification-with-lstm
   - https://www.dataknowsall.com/multiclass.html

